In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [2]:
from google.colab import auth
auth.authenticate_user() # Authenticate to the cloud storage

In [3]:
# URLs of a few tiles.
urls = [
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-124.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-124.tif'
]

# You need to have write access to this bucket.
bucket = 'tpu-aakash'

In [ ]:
# Pipe curl output to gsutil.
for f in urls:
  filepath = bucket + '/' + f.split('/')[-1]
  !curl {f} | gsutil cp - {filepath}

The above command didn't upload it to the bucket 

So I first created a folder (gee-folder) and then I am going to check if it works 

In [10]:
import os 

filelist = os.listdir('tpu-aakash')
#filelist

['fromglc10v01_36_-120.tif',
 'fromglc10v01_36_-122.tif',
 'fromglc10v01_38_-122.tif',
 'fromglc10v01_36_-124.tif',
 'fromglc10v01_38_-124.tif',
 'fromglc10v01_38_-120.tif']

In [ ]:
%%time
for file in filelist:
  !gsutil cp tpu-aakash/{file} gs://tpu-aakash/gee-folder/

Build the manifest file which is similar to the dataset-metajson file that you create before uploading a dataset to kaggle


In [16]:
!gsutil ls gs://{bucket}

gs://tpu-aakash/DrugVisData.csv
gs://tpu-aakash/DrugVisData_small.csv
gs://tpu-aakash/Image_Gain.png
gs://tpu-aakash/Image_Split.png
gs://tpu-aakash/us-states.txt
gs://tpu-aakash/Access GCP bucket/
gs://tpu-aakash/bert-checkpoints/
gs://tpu-aakash/bert-tfhub/
gs://tpu-aakash/face-crop-v1/
gs://tpu-aakash/face-crop-v2/
gs://tpu-aakash/fake_butPred_real/
gs://tpu-aakash/gee-folder/
gs://tpu-aakash/models/
gs://tpu-aakash/real_butPred_fake/
gs://tpu-aakash/resnet-2x/


In [17]:
# List the contents of the cloud storage bucket 
cloud_files = !gsutil ls gs://{bucket}'/gee-folder/*.tif'

# Get the list of source URIs from the gsutil output.
sources_uris = [{'uris': [f]} for f in cloud_files]

In [18]:
print(cloud_files)

['gs://tpu-aakash/gee-folder/fromglc10v01_36_-120.tif', 'gs://tpu-aakash/gee-folder/fromglc10v01_36_-122.tif', 'gs://tpu-aakash/gee-folder/fromglc10v01_36_-124.tif', 'gs://tpu-aakash/gee-folder/fromglc10v01_38_-120.tif', 'gs://tpu-aakash/gee-folder/fromglc10v01_38_-122.tif', 'gs://tpu-aakash/gee-folder/fromglc10v01_38_-124.tif']


In [19]:
print(sources_uris)

[{'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_36_-120.tif']}, {'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_36_-122.tif']}, {'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_36_-124.tif']}, {'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_38_-120.tif']}, {'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_38_-122.tif']}, {'uris': ['gs://tpu-aakash/gee-folder/fromglc10v01_38_-124.tif']}]


In [41]:
asset_name = 'projects/earthengine-legacy/assets/users/aakashg80/gaia_manifest'

# The enclosing object for the asset.
asset = {
  'name': asset_name,
  'tilesets': [
    {
      'sources': sources_uris
    }
  ],
  'bands': [
    {
      'id': 'cover_code',
      'pyramiding_policy': 'MODE',
      'missing_data': {
          'values': [0]
      }
    }
  ]
}

#import json  # printout the json file to check if its written properly
#print(json.dumps(asset, indent=2))

In [42]:
# Print the file 
file_name = 'gaia_manifest.json'

with open(file_name, 'w') as f:
  json.dump(asset, f, indent=2)



In [43]:
# Check the file for errors 
#!cat {file_name}

The next command for uploading the tiles, didn't work. 

First - I couldn't figure the correct path_name in the manifest file 
        This was resolved. I need to point to the asset folder (check the engine platform)

Second - It still giving me some error - *No module named google.appengine* & *PERMISSION DENIED*

So checked the CLI options, and then ran the command for authenticating it from the command line 

```
$ earthengine authenticate 
```

Still the authentication error

tried uploading a single file insted of using the manifest, 

but that also gave the same error. Command that I used 

```
$ earthengine upload image --asset_id=users/aakashg80/gaia_manifest gs://bucket/filename.tiff
```
Then I tried uploading from the UI interface 

And visualized it with folium - (the code blocks that didin't work follow the vizualizaion)

In [57]:
# Use a modified FROM-GLC palette to visualize the results.

palette = [
    'a3ff73', # farmland
    '267300', # forest
    'ffff00', # grassland
    '70a800', # shrub
    '00ffff', # wetland
    '005cff', # water
    '004600', # tundra
    'c500ff', # impervious
    'ffaa00', # bare
    'd1d1d1', # snow, ice
]
vis = {'min': 10, 'max': 100, 'palette': palette}


In [59]:
ingested_image = ee.Image('users/aakashg80/gaia_manifest/fromglc10v01_36_-120')
map_id = ingested_image.getMapId(vis)

In [60]:
import folium

map = folium.Map(location=[37.6413, -122.2582])
folium.TileLayer(
  tiles=map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='fromglc10_demo',
).add_to(map)
map.add_child(folium.LayerControl())
map

**The following code didn't work!** :( 

In [53]:
# Do the upload. Earth Engine was initialized in the first code block
#!earthengine upload image --manifest {file_name}

In [54]:
#!earthengine authenticate

In [55]:
#!earthengine acl get users/aakashg80/gaia_manifest

In [56]:
#!earthengine upload image --asset_id=users/aakashg80/gaia_manifest gs://tpu-aakash/gee-folder/fromglc10v01_36_-120.tif